Chargement et nettoyages des données

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly_express as px
import numpy as nu
from sklearn.tree import DecisionTreeRegressor

train = pd.read_csv('data/train.csv')



Repérage des valeurs manquantes

In [3]:
train_isnull = train.isnull().sum()
print(train_isnull)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


Détection des entrées manquantes groupés par la Pclass, Sexe et Parch

In [4]:
train[train['Age'].isnull()].groupby(['Pclass', 'Sex', 'Parch']).size()

Pclass  Sex     Parch
1       female  0         8
                1         1
        male    0        21
2       female  0         2
        male    0         9
3       female  0        31
                1         4
                2         7
        male    0        86
                1         3
                2         5
dtype: int64

Extraction des titres dans la colonne 'Name'

In [5]:
train['Title'] = train['Name'].str.extract(r',\s*([^\.]*)\s*\.', expand=False)
train['Title'] = train['Title'].str.strip()

train[train['Age'].isnull()].groupby(['Title', 'Pclass', 'Sex', 'Parch']).size()


Title   Pclass  Sex     Parch
Dr      1       male    0         1
Master  3       male    1         3
                        2         1
Miss    1       female  0         1
        2       female  0         2
        3       female  0        24
                        1         4
                        2         5
Mr      1       male    0        20
        2       male    0         9
        3       male    0        86
                        2         4
Mrs     1       female  0         7
                        1         1
        3       female  0         7
                        2         2
dtype: int64

Calcul de la médiane

In [6]:
age_medians = train.groupby(['Title', 'Pclass'])['Age'].median()
# print(age_medians)


Création d'une fonction pour attribuer l'âge correctement en fonction du titre, age et classe de voyageur

In [7]:
def impute_age(row):
    if pd.isnull(row['Age']):
        return age_medians.get((row['Title'], row['Pclass']), train['Age'].median())
    return row['Age']


Application de la fonction sur les données manquantes

In [8]:
train['Age'] = train.apply(impute_age, axis=1)
# Verification
print("Valeurs manquantes restantes :", train['Age'].isnull().sum())


Valeurs manquantes restantes : 0


Histogramme de la répartition des passagers en fonction de leur âge et sexe

In [14]:
import plotly.express as px

# Histogramme interactif de la distribution des âges
fig = px.histogram(
    train,
    x="Age",
    color="Sex",
    nbins=30,
    marginal="box",   # ou "violin" ou "rug" pour ajouter un petit résumé
    title="Distribution des âges après imputation (par sexe)",
    opacity=0.7
)

fig.update_layout(bargap=0.1)
fig.show()


Visualisation des données

In [10]:
fig = px.histogram(
    train,
    x="Pclass",
    color="Survived", 
    barmode="overlay",  
    facet_col="Sex",
    nbins=30,
    title="Distribution de l'âge selon le sexe et la survie"
)
fig.show()

Distribution de l'âge en fonction du titre

In [13]:
fig = px.histogram(train,
             x='Title',
             y='Age',
             color='Sex',
             title='Distribution de l\'âge selon le titre')
fig.show()